In [1]:

# Từ bây giờ ta sẽ quan tấm chủ yếu tới những đối tượng có nguy cơ 
# Lấy ra danh sách người có nguy cơ
import pandas as pd

csv_file = 'data/data_da_chuyen_doi.csv'
df = pd.read_csv(csv_file)

# Tạo một từ điển để theo dõi đường liên kết và số lần xuất hiện
link_count = {}

# Duyệt qua từng dòng trong DataFrame
for index, row in df.iterrows():
    suicide_label = row['Suicide']
    
    # Kiểm tra xem cột "Suicide" có nhãn là 1 không
    if suicide_label == 1:
        link_facebook = row['link_facebook']
        
        # Kiểm tra xem liên kết đã xuất hiện trước đó trong từ điển chưa
        if link_facebook in link_count:
            link_count[link_facebook] += 1
        else:
            link_count[link_facebook] = 1

# Chuyển từ điển thành danh sách 2 chiều
result = [[link, count] for link, count in link_count.items()]

# In danh sách kết quả ra màn hình
for item in result:
    print(f"Link: {item[0]}, Số lần xuất hiện: {item[1]}")


Link: https://www.facebook.com/profile.php?id=61550489318359, Số lần xuất hiện: 2


Khi ta đã có danh sách đường link của đối tượng có nguy cơ và số lần xuất hiện bài viết có nguy cơ. Ta sẽ có thể tiến hành theo dõi đánh giá kĩ hơn các đối tượng này . Nghiện cứu thêm về cách tính trọng số đánh giá mức độ trầm cảm sẽ là một hướng đi tốt. ý tưởng hiện tại : dựa vào số bài viết có nguy cơ , tần suất xuất hiện của chúng , kết hợp với dùng ngưỡng phần loại threshold của model để kết hợp tính toán. Từ đó làm căn cứ phát đi cảnh báo sau này. Để có thể làm vậy thì ta sẽ thu thập thêm dữ liệu từ bạn bè.


In [19]:
# Lấy dữ liệu bạn bè
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import random
import pyautogui as pg
import time
import io
import pyperclip
import csv
from bs4 import BeautifulSoup

# Kiểm tra xem result có chứa các đường liên kết không
if len(result) == 0:
    print("Không có đường liên kết nào có cột 'Suicide' bằng 1. Result rỗng.")
else:
    # Tạo đối tượng Options và cấu hình đường dẫn đến Chrome WebDriver
    chrome_options = Options()
    chrome_options.binary_location = "chromedriver-win64/chromedriver.exe"  
    # Không hiển thị thông báo
    chrome_options.add_argument("--disable-notifications")  
    # Tạo đối tượng WebDriver
    browser = webdriver.Chrome(options=chrome_options)

    # Mở trang web Facebook
    browser.get("http://facebook.com")
    sleep(random.randint(3, 7))

    # Điền thông tin đăng nhập và đăng nhập vào tài khoản
    txtUser = browser.find_element("name", "email")
    txtUser.send_keys("buidang0703@gmail.com")
    sleep(random.randint(1, 4))
    txtPass = browser.find_element("name", "pass")
    txtPass.send_keys("Anhfangpro@")

    txtPass.send_keys(Keys.ENTER)
    sleep(random.randint(3, 5))

    # Duyệt qua từng đường liên kết trong danh sách kết quả
    for item in result:
        link = item[0]
        count = item[1]

        # Mở trình duyệt và truy cập vào liên kết
        browser.get(link)
        sleep(random.randint(3, 5))

        csv_file_name = 'data/doi_tuong_can_theo_doi.csv'
        # Mở file CSV để đọc dữ liệu
        existing_data = []
        with open(csv_file_name, 'r', newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                existing_data.append(row['link_facebook_doi_tuong'])

        # Kiểm tra xem đường link đã có trong danh sách chưa
        if link not in existing_data:
            # Mở trình duyệt và truy cập vào đường link Facebook của đối tượng
            browser.get(link)
            sleep(random.randint(3, 5))

            # Tạo một dòng dữ liệu mới
            row_data = {'link_facebook_doi_tuong': link, 'Friend1': '', 'Friend2': '', 'Friend3': '', 'Friend4': '', 'Friend5': ''}

            # Lấy danh sách các đường link bạn bè
            friend_links = browser.find_elements(by="css selector", value=".x1i10hfl.xjbqb8w.x6umtig.x1b1mbwd.xaqea5y.xav7gou.x9f619.x1ypdohk.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1o1ewxj.x3x9cwd.x1e5q0jg.x13rtm0m.x87ps6o.x1lku1pv.x1a2a7pz.x1lliihq.x1vrad04.x1n2onr6.xh8yej3")

            # Lặp qua danh sách bạn bè và lưu vào các cột Friend
            for i, linkf in enumerate(friend_links[:5], start=1):
                soup = BeautifulSoup(linkf.get_attribute('outerHTML'), 'html.parser')
                link_element = soup.find('a', href=True)
                if link_element:
                    friend_link = link_element['href']
                    row_data[f'Friend{i}'] = friend_link

            # Ghi dòng dữ liệu vào file CSV
            with open(csv_file_name, 'a', newline='', encoding='utf-8') as csvfile:
                fieldnames = ['link_facebook_doi_tuong', 'Friend1', 'Friend2', 'Friend3', 'Friend4', 'Friend5']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow(row_data)
        else:
            print("Đường link đã tồn tại trong danh sách.")

browser.quit()   

